In [3]:
import datetime
import time as time_module
import sys
import os 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import theano
import matplotlib
import pymc3 as pm
import theano.tensor as tt



try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('..')
    import covid19_inference as cov19
%load_ext autoreload
%autoreload 2


In [4]:
#df_rki = cov19.data_retrieval.get_rki()
import pickle
#pickle.dump(df_rki, open('df_rki.pickled', 'wb'))
df_rki = pickle.load(open('df_rki.pickled', 'rb'))

In [10]:
df_bundeslaender = cov19.filter_rki_all_bundesland(df_rki, '2020-03-10', '2020-04-13')
new_cases_obs = np.diff(np.array(df_bundeslaender),axis=0)[:,:]


date_begin_data = datetime.datetime(2020,3,10)
date_end_data   = datetime.datetime(2020,4,13)

#date_data_end   = datetime.datetime(2020,3,28)
num_days_data = (date_end_data-date_begin_data).days
diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_forecast = 10
date_begin_sim = date_begin_data - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_end_data   + datetime.timedelta(days = num_days_forecast)
num_days_sim = (date_end_sim-date_begin_sim).days


#print('Cases yesterday ({}): {} and '
#      'day before yesterday: {}'.format(date_data_end.isoformat(), *cases_obs[:-3:-1]))

prior_date_mild_dist_begin =  datetime.datetime(2020,3,9)
prior_date_strong_dist_begin =  datetime.datetime(2020,3,16)
prior_date_contact_ban_begin =  datetime.datetime(2020,3,23)

change_points = [dict(pr_mean_date_transient = prior_date_mild_dist_begin,
                      pr_sigma_date_transient = 6,
                      pr_median_lambda = 0.2,
                      pr_sigma_lambda = 0.5),
                 dict(pr_mean_date_transient = prior_date_strong_dist_begin,
                      pr_sigma_date_transient = 6,
                      pr_median_lambda = 1/8,
                      pr_sigma_lambda = 0.5),
                 dict(pr_mean_date_transient = prior_date_contact_ban_begin,
                      pr_sigma_date_transient = 6,
                      pr_median_lambda = 1/8/2,
                      pr_sigma_lambda = 0.5)]

In [13]:
params_model = dict(new_cases_obs = new_cases_obs,
                    date_begin_data = date_begin_data,
                    num_days_forecast = num_days_forecast,
                    diff_data_sim = diff_data_sim,
                    N_population = 83e6)

with cov19.Cov19_Model(**params_model) as model:
    
    lambda_t_log = cov19.lambda_t_with_sigmoids(pr_median_lambda_0 = 0.4,
                                                change_points_list = change_points)
    
    new_I_t = cov19.SIR(lambda_t_log)
    
    new_cases_inferred = cov19.delay_cases(new_I_t)
    
    new_cases_inferred = cov19.week_modulation(new_cases_inferred)
    
    cov19.student_t_likelihood(new_cases_inferred)
    
    
    
    

In [ ]:
trace = pm.sample(model = model)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_obs, offset_modulation_rad, weekend_factor_L2, weekend_factor_L1, sigma_weekend_factor_L2, scale_delay_L2, scale_delay_L1, sigma_scale_delay_L2, delay_L2, delay_L1, sigma_delay_L2, I_begin, mu, transient_len_3_L2, transient_len_3_L1, sigma_transient_len_3_L2, transient_len_2_L2, transient_len_2_L1, sigma_transient_len_2_L2, transient_len_1_L2, transient_len_1_L1, sigma_transient_len_1_L2, transient_day_3_L2, transient_day_3_L1, sigma_transient_day_3_L2, transient_day_2_L2, transient_day_2_L1, sigma_transient_day_2_L2, transient_day_1_L2, transient_day_1_L1, sigma_transient_day_1_L2, lambda_3_L2, lambda_3_L1, sigma_lambda_3_L2, lambda_2_L2, lambda_2_L1, sigma_lambda_2_L2, lambda_1_L2, lambda_1_L1, sigma_lambda_1_L2, lambda_0_L2_L2, lambda_0_L2_L1, sigma_lambda_0_L2_L2]
Sampling 2 chains:  23%|██▎       | 461/2000 [05:23<47:31,  1.85s/draws]  